# Test Metrics on Model

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import change_dataset_np
from torchvision import datasets, models, transforms
import seg_metrics
import models
from PIL import Image
import matplotlib.pyplot as plt
img_size = 224
num_classes = 2
batch_size = 1
val_pickle_file = 'change_dataset_train.pkl'

from IPython.display import clear_output, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of available GPUs:', num_gpu)

Device: cuda:0
Number of available GPUs: 8


In [8]:
#transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

In [9]:
val_dataset = change_dataset_np.ChangeDatasetNumpy(val_pickle_file, data_transforms['val'])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

#### Initialize Model and Load Checkpoint

In [4]:
change_net = models.ChangeNet(num_classes=num_classes)
if num_gpu > 1:
    change_net = nn.DataParallel(change_net)
change_net = change_net.to(device)

checkpoint = torch.load('./best_model-NoRandCrop.pkl')
change_net.load_state_dict(checkpoint);
change_net.eval();

In [10]:
iteractive_idx = 0
output = 0
label_img = 0
preds = 0
@interact(idx=widgets.IntSlider(min=0,max=len(val_dataset)-1))
def explore_validation_dataset(idx):
    global iteractive_idx
    global output
    global label_img
    global preds
    sample = val_dataset[idx]
    reference_img = sample['reference']
    test_img = sample['test']
    label_img = sample['label']
    preds = change_net([reference_img.unsqueeze(0), test_img.unsqueeze(0)])
    _, output = torch.max(preds, 1)
    output = output.unsqueeze(0)
    print(output.shape)
    plt.imshow(reference_img.permute(1, 2, 0).numpy())
    plt.show()
    plt.imshow(test_img.permute(1, 2, 0).numpy())
    plt.show()
    plt.imshow(label_img.squeeze(0).numpy())
    plt.show()
    plt.imshow(output.squeeze(0).squeeze(0).cpu().numpy())
    plt.show()
    iteractive_idx = idx

interactive(children=(IntSlider(value=0, description='idx', max=73275), Output()), _dom_classes=('widget-inter…

In [11]:
print('output.shape:', output.shape)
print(torch.max(output).item())
print('output type:', output.type())

print('preds.shape:', preds.shape)
print(torch.max(preds).item())
print('preds type:', preds.type())

print('label_img.shape:', label_img.shape)
print(torch.max(label_img).item())
print('label_img type:', label_img.type())

output.shape: torch.Size([1, 1, 224, 224])
1
output type: torch.cuda.LongTensor
preds.shape: torch.Size([1, 2, 224, 224])
10.89464282989502
preds type: torch.cuda.FloatTensor
label_img.shape: torch.Size([1, 224, 224])
0.003921568859368563
label_img type: torch.FloatTensor


In [12]:
iou = seg_metrics.iou_segmentation(output.squeeze(1).to(device).type(torch.LongTensor), (label_img>0).to(device).type(torch.LongTensor))
#iou_pytorch(output.squeeze(1).to(device).type(torch.uint8), output.squeeze(1).to(device).type(torch.uint8))
print(iou)

tensor(0.9692)
